<a href="https://colab.research.google.com/github/HarshalGoyal/Intent_detection_nlp/blob/main/intent_detection_algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q scikit-learn pandas numpy matplotlib seaborn
!pip install -q torch torchvision torchaudio
!pip install -q transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from datasets import load_dataset
clinc = load_dataset("clinc_oos", "plus")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/24.0k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/312k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/136k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15250 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3100 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'intent'],
        num_rows: 15250
    })
    validation: Dataset({
        features: ['text', 'intent'],
        num_rows: 3100
    })
    test: Dataset({
        features: ['text', 'intent'],
        num_rows: 5500
    })
})
{'text': 'what expression would i use to say i love you if i were an italian', 'intent': 61}


In [4]:
print(clinc)
print(clinc["train"][0])

DatasetDict({
    train: Dataset({
        features: ['text', 'intent'],
        num_rows: 15250
    })
    validation: Dataset({
        features: ['text', 'intent'],
        num_rows: 3100
    })
    test: Dataset({
        features: ['text', 'intent'],
        num_rows: 5500
    })
})
{'text': 'what expression would i use to say i love you if i were an italian', 'intent': 61}


In [39]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag, word_tokenize
from nltk.corpus import stopwords
import nltk
import re

nltk.download('all')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)

    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t not in stop_words]
    pos_tags = pos_tag(tokens)
    lemmatized = [
        lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags
    ]

    return ' '.join(lemmatized)

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pickle

In [54]:
def train_traditional_probablistic_model (X_train, y_train, X_test, y_test,
                                          tune_model = False, model_params = {'alpha': [0.1, 0.5, 1.0]},
                                          vectorizer_params = {'ngram_range': (1, 2), 'min_df': 3, 'max_df': 0.9},
                                          cross_val = True):

  X_train_clean = [preprocess_text(text) for text in X_train]
  X_test_clean  = [preprocess_text(text) for text in X_test]

  vectorizer_params = vectorizer_params or {}
  vectorizer = TfidfVectorizer(**vectorizer_params)
  X_train_vec = vectorizer.fit_transform(X_train_clean)
  X_test_vec = vectorizer.transform(X_test_clean)
  model = MultinomialNB()

  if tune_model:
    model_parameters = model_params
    grid = GridSearchCV(model, model_parameters, cv=5, n_jobs=-1, scoring='accuracy')
    grid.fit (X_train_vec, y_train)
    model = grid.best_estimator_
    print(f"Best model params: {grid.best_params_}")

  else:
    model.fit (X_train_vec, y_train)

  if cross_val:
        from sklearn.model_selection import cross_val_score
        cv_scores = cross_val_score(model, X_train_vec, y_train, cv=5, scoring='accuracy')
        print(f"Cross-validation scores: {cv_scores}")
        print(f"Average cross-validation score: {cv_scores.mean():.4f}")

  y_pred = model.predict (X_test_vec)
  acc = accuracy_score(y_test, y_pred)
  f1 = f1_score(y_test, y_pred, average='weighted')
  precision = precision_score(y_test, y_pred, average='weighted')
  recall = recall_score(y_test, y_pred, average='weighted')

  vectorizer_params_str = '_'.join([f'{k}{str(v).replace(".", "_")}' for k, v in vectorizer_params.items()])
  model_params_str = f"alpha_{str(model.alpha).replace('.', '_')}" if hasattr(model, 'alpha') else "default"
  acc_str = f"acc_{int(acc * 100)}"
  f1_str = f"f1_{int(f1 * 100)}"

  model_filename = f"/content/intent_model_{model_params_str}_{acc_str}_{f1_str}.pkl"
  vectorizer_filename = f"/content/tfidf_vectorizer_{vectorizer_params_str}.pkl"


  with open(model_filename, 'wb') as f:
        pickle.dump(model, f)

  with open(vectorizer_filename, 'wb') as f:
        pickle.dump(vectorizer, f)

  print(f"Model saved with Accuracy: {acc * 100:.2f}% | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f}")
  print(f"Model saved as {model_filename} and vectorizer as {vectorizer_filename}")

In [55]:
X_train = clinc['train']['text']
y_train = clinc['train']['intent']

X_val = clinc['validation']['text']
y_val = clinc['validation']['intent']

X_test = clinc['test']['text']
y_test = clinc['test']['intent']

In [56]:
train_traditional_probablistic_model (X_train,y_train,X_test,y_test)

Cross-validation scores: [0.81868852 0.81672131 0.83278689 0.83737705 0.8252459 ]
Average cross-validation score: 0.8262
Model saved with Accuracy: 76.69% | F1: 0.7578 | Precision: 0.7795 | Recall: 0.7669
Model saved as /content/intent_model_alpha1_0_acc_76_f1_75.pkl and vectorizer as /content/tfidf_vectorizer_ngram_range(1, 2)_min_df3_max_df0_9.pkl


In [59]:
def traditional_probablistic_model_predictions (input_texts, model_filepath, vectorizer_filepath):

    with open(model_filepath, 'rb') as f:
        model = pickle.load(f)
    with open(vectorizer_filepath, 'rb') as f:
        vectorizer = pickle.load(f)
    input_texts_clean = [preprocess_text(text) for text in input_texts]
    input_vec = vectorizer.transform(input_texts_clean)
    predictions = model.predict(input_vec)
    return predictions

In [60]:
predictions = traditional_probablistic_model_predictions (X_val,"/content/intent_model_alpha1_0_acc_76_f1_75.pkl", "/content/tfidf_vectorizer_ngram_range(1, 2)_min_df3_max_df0_9.pkl")

In [61]:
accuracy = accuracy_score (y_val, predictions)
f1 = f1_score(y_val, predictions, average="weighted")
precision = precision_score (y_val, predictions, average="weighted")
recall = recall_score (y_val, predictions, average="weighted")

In [62]:
print(f"Validation Accuracy: {accuracy * 100:.2f}%")
print(f"Validation F1: {f1:.4f}")
print(f"Validation Precision: {precision:.4f}")
print(f"Validation Recall: {recall:.4f}")

Validation Accuracy: 80.87%
Validation F1: 0.8058
Validation Precision: 0.8233
Validation Recall: 0.8087


In [64]:
intent_labels = clinc['train'].features['intent'].names

In [70]:
def traditional_probablistic_model_predictions_with_labels(input_texts, model_filepath, vectorizer_filepath, intent_labels = clinc['train'].features['intent'].names):
    predictions = traditional_probablistic_model_predictions(input_texts, model_filepath, vectorizer_filepath)
    predicted_intents = [intent_labels[label] for label in predictions]
    for text, intent in zip(input_texts, predicted_intents):
        print(f"Text: {text} -> Predicted Intent: {intent}")

In [75]:
input_texts = [
    "What is the weather like today?",
    "ugh, you don't understand anything!",
    "How do I make a reservation?",
    "Ok Great!! it was really fun wasting my time teacing the same things to you for straigt 5 hours"
]

traditional_probablistic_model_predictions_with_labels (input_texts,"/content/intent_model_alpha1_0_acc_76_f1_75.pkl", "/content/tfidf_vectorizer_ngram_range(1, 2)_min_df3_max_df0_9.pkl")

Text: What is the weather like today? -> Predicted Intent: weather
Text: ugh, you don't understand anything! -> Predicted Intent: account_blocked
Text: How do I make a reservation? -> Predicted Intent: restaurant_reservation
Text: Ok Great!! it was really fun wasting my time teacing the same things to you for straigt 5 hours -> Predicted Intent: what_are_your_hobbies
